# 层和块
回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2104,  0.0102, -0.2692,  0.0847, -0.0219, -0.0952,  0.1185, -0.0125,
         -0.1051,  0.0842],
        [ 0.1741,  0.0097, -0.1103, -0.0171, -0.0202, -0.2051,  0.0967, -0.0929,
         -0.1254,  0.1742]], grad_fn=<AddmmBackward0>)

自定义类

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()#调用父类的初始化
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self,x):
        return self.out(F.relu(self.hidden(x)))

实例化多层感知机

In [8]:
net = MLP()
net(X)

tensor([[-0.0769, -0.2405,  0.0901,  0.0908,  0.0519,  0.0228, -0.0103,  0.0335,
         -0.0768,  0.0627],
        [-0.1329, -0.0997,  0.0721,  0.0366, -0.0410, -0.0086,  0.0506, -0.0172,
         -0.0947,  0.0748]], grad_fn=<AddmmBackward0>)

顺序块

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1135,  0.0799, -0.1876, -0.0242,  0.0363, -0.0275,  0.0530, -0.1528,
          0.0425,  0.0302],
        [ 0.1175, -0.0742, -0.1830, -0.1238,  0.0255, -0.1616,  0.0127, -0.2473,
          0.1911,  0.0455]], grad_fn=<AddmmBackward0>)

nn.Module初始化后创造出的Net必须有的字典属性：  
_parameters  
_modules  
_buffers  
_backward_hooks  
_forward_hooks  
_forward_pre_hooks  
_state_dict_hooks  
_load_state_dict_pre_hooks  

在正向传播代码中执行代码

In [15]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.0049, grad_fn=<SumBackward0>)

混合搭配各种组合块

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0202, grad_fn=<SumBackward0>)